In [1]:
import project_functions as pf
import numpy as np
import pandas as pd
import catboost as ctb
import lightgbm as lgb
# import xgboost as xgb

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

In [2]:
def create_data_set(train: pd.DataFrame, path: str) -> pd.DataFrame:
    # PERS
    df = pd.read_csv(path + 'client_profile.csv')   
    #----------------------------------------------------------------------------
    df['DAYS_ON_LAST_JOB'] = df['DAYS_ON_LAST_JOB'].replace(365243, np.nan)
    bki_flag = [flag  for flag in df.columns if 'AMT_REQ_CREDIT_BUREAU' in flag]
    df['BKI_REQUEST_COUNT'] = df[bki_flag].sum(axis=1)
    df['BKI_KURTOSIS'] = df[bki_flag].kurtosis(axis=1)
    df['EXTERNAL_SCORING_PROD'] = df['EXTERNAL_SCORING_RATING_1'] * df['EXTERNAL_SCORING_RATING_2'] * df['EXTERNAL_SCORING_RATING_3']
    df['EXTERNAL_SCORING_WEIGHT'] = df['EXTERNAL_SCORING_RATING_1'] * 2 + df['EXTERNAL_SCORING_RATING_2'] * 1 + df['EXTERNAL_SCORING_RATING_3'] * 3
    
    for function_name in ['min', 'max', 'mean', 'nanmedian', 'var']:
        feature_name = f'EXTERNAL_SCORING_RATING_{function_name}'
        df[feature_name] = eval(f'np.{function_name}')(
            df[['EXTERNAL_SCORING_RATING_1', 'EXTERNAL_SCORING_RATING_2', 'EXTERNAL_SCORING_RATING_3']], 
            axis=1
        )
        
    df['ratio_credit_to_annuity'] = df['AMOUNT_CREDIT'] / df['AMOUNT_ANNUITY']
    df['ratio_annuity_to_salary'] = df['AMOUNT_ANNUITY'] / df['TOTAL_SALARY']
    df['ratio_credit_to_salary'] = df['AMOUNT_CREDIT'] / df['TOTAL_SALARY']
    
    df['ratio_credit_to_age'] = df['AMOUNT_CREDIT'] / df['AGE']
    df['ratio_annuity_to_age'] = df['AMOUNT_ANNUITY'] / df['AGE']
    df['ratio_salary_to_age'] = df['TOTAL_SALARY'] / df['AGE']
    df['ratio_credit_to_job'] = df['AMOUNT_CREDIT'] / df['DAYS_ON_LAST_JOB']
    df['ratio_annuity_to_job'] = df['AMOUNT_ANNUITY'] / df['DAYS_ON_LAST_JOB']
    df['ratio_credit_to_job'] = df['TOTAL_SALARY'] / df['DAYS_ON_LAST_JOB']
    
    df['ratio_age_to_job'] = df['AGE'] / df['DAYS_ON_LAST_JOB']
    df['ratio_salary_to_population'] = df['TOTAL_SALARY'] / df['REGION_POPULATION']
    df['ratio_cat_to_job'] = df['OWN_CAR_AGE'] / df['DAYS_ON_LAST_JOB']
    df['ratio_car_to_age'] = df['OWN_CAR_AGE'] / df['AGE']
    
    df['loss_1'] = df['EXTERNAL_SCORING_RATING_1'] * df['AMOUNT_CREDIT']
    df['loss_2'] = df['EXTERNAL_SCORING_RATING_2'] * df['AMOUNT_CREDIT']
    df['loss_3'] = df['EXTERNAL_SCORING_RATING_3'] * df['AMOUNT_CREDIT']
    df['loss_4'] = df['EXTERNAL_SCORING_RATING_1'] * df['AMOUNT_ANNUITY']
    df['loss_5'] = df['EXTERNAL_SCORING_RATING_2'] * df['AMOUNT_ANNUITY']
    df['loss_6'] = df['EXTERNAL_SCORING_RATING_3'] * df['AMOUNT_ANNUITY']    
    
    #----------------------------------------------------------------------------

    df.loc[df['FAMILY_SIZE'].isna(), 'FAMILY_SIZE'] = 1
    df['EXISTS_CAR'] = df['OWN_CAR_AGE'].isna().astype(int)
    df.drop(columns=['OWN_CAR_AGE'], inplace=True)
    df.fillna({'EXTERNAL_SCORING_RATING_1': 0,
               'EXTERNAL_SCORING_RATING_2': 0,
               'EXTERNAL_SCORING_RATING_3': 0},
              inplace=True)
    df['EXTERNAL_SCORING_RATING_MEAN'] = df.apply(lambda x: pf.calc_mean_scoring_rating(x['EXTERNAL_SCORING_RATING_1'], 
                                                                                    x['EXTERNAL_SCORING_RATING_2'],
                                                                                    x['EXTERNAL_SCORING_RATING_3']), 
                                                  axis=1)
    df['EXIST_CREDIT_BUREAU_INFORMATION'] = df['AMT_REQ_CREDIT_BUREAU_HOUR'].isna().astype(int)
    df.fillna({'AMT_REQ_CREDIT_BUREAU_HOUR': -1,
               'AMT_REQ_CREDIT_BUREAU_DAY': -1,
               'AMT_REQ_CREDIT_BUREAU_WEEK': -1,
               'AMT_REQ_CREDIT_BUREAU_MON': -1,
               'AMT_REQ_CREDIT_BUREAU_QRT': -1,
               'AMT_REQ_CREDIT_BUREAU_YEAR': -1,
              },
              inplace=True)

    gender = ['F', 'M', 'XNA']
    df.loc[df['GENDER'].isin(gender) == False, 'GENDER'] = 3
    df.loc[df['GENDER'] == gender[0], 'GENDER'] = 1
    df.loc[df['GENDER'] == gender[1], 'GENDER'] = 2
    df.loc[df['GENDER'] == gender[2], 'GENDER'] = 2
    df['GENDER'] = df['GENDER'].astype(int)

    education_levels = ['Secondary / secondary special', 'Higher education', 'Incomplete higher', 
                        'Lower secondary', 'Academic degree']
    df.loc[df['EDUCATION_LEVEL'].isin(education_levels) == False, 'EDUCATION_LEVEL'] = 6
    df.loc[df['EDUCATION_LEVEL'] == education_levels[0], 'EDUCATION_LEVEL'] = 1
    df.loc[df['EDUCATION_LEVEL'] == education_levels[1], 'EDUCATION_LEVEL'] = 2
    df.loc[df['EDUCATION_LEVEL'] == education_levels[2], 'EDUCATION_LEVEL'] = 3
    df.loc[df['EDUCATION_LEVEL'] == education_levels[3], 'EDUCATION_LEVEL'] = 4
    df.loc[df['EDUCATION_LEVEL'] == education_levels[4], 'EDUCATION_LEVEL'] = 5
    df['EDUCATION_LEVEL'] = df['EDUCATION_LEVEL'].astype(int)

    family_status = ['Married', 'Single / not married', 'Civil marriage', 'Separated', 'Widow', 'Unknown']
    df.loc[df['FAMILY_STATUS'].isin(family_status) == False, 'FAMILY_STATUS'] = 6
    df.loc[df['FAMILY_STATUS'] == family_status[0], 'FAMILY_STATUS'] = 1
    df.loc[df['FAMILY_STATUS'] == family_status[1], 'FAMILY_STATUS'] = 2
    df.loc[df['FAMILY_STATUS'] == family_status[2], 'FAMILY_STATUS'] = 3
    df.loc[df['FAMILY_STATUS'] == family_status[3], 'FAMILY_STATUS'] = 4
    df.loc[df['FAMILY_STATUS'] == family_status[4], 'FAMILY_STATUS'] = 5
    df.loc[df['FAMILY_STATUS'] == family_status[5], 'FAMILY_STATUS'] = 1
    df['FAMILY_STATUS'] = df['FAMILY_STATUS'].astype(int)

    df.loc[(df['CHILDRENS'] == 1) | (df['CHILDRENS'] == 2) | (df['CHILDRENS'] == 3), 'CHILDRENS'] = 1
    df.loc[(df['CHILDRENS'] == 4) | (df['CHILDRENS'] == 5), 'CHILDRENS'] = 2
    df.loc[(df['CHILDRENS'] == 6), 'CHILDRENS'] = 3
    df.loc[(df['CHILDRENS'] > 6), 'CHILDRENS'] = 4

    contract_ytype = ['Cash', 'Credit Card']
    train.loc[train['NAME_CONTRACT_TYPE'].isin(contract_ytype) == False, 'NAME_CONTRACT_TYPE'] = 3
    train.loc[train['NAME_CONTRACT_TYPE'] == contract_ytype[0], 'NAME_CONTRACT_TYPE'] = 1 
    train.loc[train['NAME_CONTRACT_TYPE'] == contract_ytype[1], 'NAME_CONTRACT_TYPE'] = 2
    train['NAME_CONTRACT_TYPE'] = train['NAME_CONTRACT_TYPE'].astype(int)

    train = train.merge(df.set_index('APPLICATION_NUMBER'), how='left', on='APPLICATION_NUMBER')
    
    # BKI
    df = pd.read_csv(path + 'bki.csv')
    
    train['EXISTS_ACTIVE_CREDIT'] = train['APPLICATION_NUMBER'].isin(df.loc[df['CREDIT_ACTIVE'] == 'Active', 
                                                                    'APPLICATION_NUMBER'].values).astype(int)
    train['EXISTS_CLOSED_CREDIT'] = train['APPLICATION_NUMBER'].isin(df.loc[df['CREDIT_ACTIVE'] == 'Closed', 
                                                                    'APPLICATION_NUMBER'].values).astype(int)


    train['EXISTS_ACTIVE_CCUR1'] = train['APPLICATION_NUMBER'].isin(df.loc[(df['CREDIT_ACTIVE'] == 'Active')
                                                                        & (df['CREDIT_CURRENCY'] == 'currency 1'), 
                                                                    'APPLICATION_NUMBER'].values).astype(int)
    train['EXISTS_ACTIVE_CCUR2'] = train['APPLICATION_NUMBER'].isin(df.loc[(df['CREDIT_ACTIVE'] == 'Active')
                                                                        & (df['CREDIT_CURRENCY'] == 'currency 2'), 
                                                                    'APPLICATION_NUMBER'].values).astype(int)
    train['EXISTS_ACTIVE_CCUR3'] = train['APPLICATION_NUMBER'].isin(df.loc[(df['CREDIT_ACTIVE'] == 'Active')
                                                                        & (df['CREDIT_CURRENCY'] == 'currency 3'), 
                                                                    'APPLICATION_NUMBER'].values).astype(int)

    train['EXISTS_OVERDUE'] = train['APPLICATION_NUMBER'].isin(df.loc[df['CREDIT_DAY_OVERDUE'] > 0, 
                                                                        'APPLICATION_NUMBER'].values
                                                                    ).astype(int)

    train['EXISTS_CREDIT_PROLONG'] = train['APPLICATION_NUMBER'].isin(
                                                                df.loc[df['CNT_CREDIT_PROLONG'] > 0, 
                                                                            'APPLICATION_NUMBER'].values
                                                                        ).astype(int)

    train['EXISTS_BKI_HISTORU'] = train['APPLICATION_NUMBER'].isin(df['APPLICATION_NUMBER'].values).astype(int)

    train = pf.merge_new_feature(train,
                             df.groupby('APPLICATION_NUMBER')['APPLICATION_NUMBER'].count(),
                             'COUNT_APPLICATION'
                            )

    train = pf.merge_new_feature(train,
                             df.loc[(df['CREDIT_ACTIVE'] == 'Closed')
                                        & (df['AMT_CREDIT_SUM_DEBT'] == 0)
                                       ].groupby('APPLICATION_NUMBER')['APPLICATION_NUMBER'].count(),
                             'COUNT_APPLICATION_CLS_1'
                            )

    train = pf.merge_new_feature(train,
                             df.loc[(df['CREDIT_ACTIVE'] == 'Closed')
                                        & (df['AMT_CREDIT_SUM_DEBT'] > 0)
                                       ].groupby('APPLICATION_NUMBER')['APPLICATION_NUMBER'].count(),
                             'COUNT_APPLICATION_CLS_2'
                            )

    train = pf.merge_new_feature(train,
                             df.loc[(df['CREDIT_ACTIVE'] == 'Closed')
                                        & (df['AMT_CREDIT_SUM_DEBT'] < 0)
                                       ].groupby('APPLICATION_NUMBER')['APPLICATION_NUMBER'].count(),
                             'COUNT_APPLICATION_CLS_3'
                            )

    train = pf.merge_new_feature(train,
                             df.loc[(df['CREDIT_ACTIVE'] == 'Closed')
                                        & (df['AMT_CREDIT_SUM_DEBT'].isna())
                                       ].groupby('APPLICATION_NUMBER')['APPLICATION_NUMBER'].count(),
                             'COUNT_APPLICATION_CLS_4'
                            )
    train['RATIO_APPLICATION_CLS_1'] = train['COUNT_APPLICATION_CLS_1'] / train['COUNT_APPLICATION']
    train['RATIO_APPLICATION_CLS_2'] = train['COUNT_APPLICATION_CLS_2'] / train['COUNT_APPLICATION']
    train['RATIO_APPLICATION_CLS_3'] = train['COUNT_APPLICATION_CLS_3'] / train['COUNT_APPLICATION']
    train['RATIO_APPLICATION_CLS_4'] = train['COUNT_APPLICATION_CLS_4'] / train['COUNT_APPLICATION']

    train = pf.merge_new_feature(train,
                             df.loc[(df['CREDIT_ACTIVE'] == 'Active')
                                        & (df['AMT_CREDIT_SUM_DEBT'] == 0)
                                       ].groupby('APPLICATION_NUMBER')['APPLICATION_NUMBER'].count(),
                             'COUNT_APPLICATION_ACT_1'
                            )

    train = pf.merge_new_feature(train,
                             df.loc[(df['CREDIT_ACTIVE'] == 'Active')
                                        & (df['AMT_CREDIT_SUM_DEBT'] > 0)
                                       ].groupby('APPLICATION_NUMBER')['APPLICATION_NUMBER'].count(),
                             'COUNT_APPLICATION_ACT_2'
                            )

    train = pf.merge_new_feature(train,
                             df.loc[(df['CREDIT_ACTIVE'] == 'Active')
                                        & (df['AMT_CREDIT_SUM_DEBT'] < 0)
                                       ].groupby('APPLICATION_NUMBER')['APPLICATION_NUMBER'].count(),
                             'COUNT_APPLICATION_ACT_3'
                            )

    train = pf.merge_new_feature(train,
                             df.loc[(df['CREDIT_ACTIVE'] == 'Active')
                                        & (df['AMT_CREDIT_SUM_DEBT'].isna())
                                       ].groupby('APPLICATION_NUMBER')['APPLICATION_NUMBER'].count(),
                             'COUNT_APPLICATION_ACT_4'
                            )

    train['RATIO_APPLICATION_ACT_1'] = train['COUNT_APPLICATION_ACT_1'] / train['COUNT_APPLICATION']
    train['RATIO_APPLICATION_ACT_2'] = train['COUNT_APPLICATION_ACT_2'] / train['COUNT_APPLICATION']
    train['RATIO_APPLICATION_ACT_3'] = train['COUNT_APPLICATION_ACT_3'] / train['COUNT_APPLICATION']
    train['RATIO_APPLICATION_ACT_4'] = train['COUNT_APPLICATION_ACT_4'] / train['COUNT_APPLICATION']

    train = pf.merge_new_feature(train,
                             df.groupby('APPLICATION_NUMBER')['AMT_CREDIT_MAX_OVERDUE'].max(),
                             'AMT_CREDIT_MAX_OVERDUE'
                            )

    tmp_df = pf.create_std_features(df[['APPLICATION_NUMBER', 'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE',
                                                    'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM_OVERDUE'
                                                   ]], 'APPLICATION_NUMBER')
    train = train.join(tmp_df.set_index('APPLICATION_NUMBER'), 
                       on='APPLICATION_NUMBER', 
                       how='left',
                       rsuffix = 'bki')
    
    train = train.replace(np.inf, np.nan)
    train = train.replace(-np.inf, np.nan)
    
    return train

In [3]:
TARGET = 'TARGET'
ID = 'APPLICATION_NUMBER'
PATH = '/home/aleksandr/data/kaggle/project/'

In [4]:
PARAM_LGB = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'n_estimators': 10000,
    'learning_rate': 0.05134,
    'early_stopping_rounds': 50,
    'reg_lambda': 0.05,
    'max_depth': 6,
    'random_state': 24
}

In [5]:
# Cтратегия валидации
cv_strategy = KFold(n_splits=5, random_state=24, shuffle=True)

In [6]:
df_train = pd.read_csv(PATH + 'train.csv')
df_train = create_data_set(df_train, PATH)

main_x = df_train.drop([TARGET], axis=1)
main_y = df_train[TARGET]
main_x = main_x.drop([ID], axis=1)

categorial = main_x.dtypes[main_x.dtypes == 'object'].index
numerycal = list(set(main_x.columns) - set(categorial))

/home/aleksandr/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)


## CTB

In [8]:
estimators_arr, oof_pred_arr, features = pf.validation(
    main_x, 
    main_y, 
    cv_strategy, 
    ctb.CatBoostClassifier, 
    PARAM_LGB, 
    roc_auc_score,
    categorial,
    []
)

0:	test: 0.6665445	test1: 0.6680111	best: 0.6680111 (0)	total: 158ms	remaining: 26m 17s
1:	test: 0.6798258	test1: 0.6757690	best: 0.6757690 (1)	total: 249ms	remaining: 20m 46s
2:	test: 0.6869247	test1: 0.6848218	best: 0.6848218 (2)	total: 386ms	remaining: 21m 25s
3:	test: 0.6965843	test1: 0.6911532	best: 0.6911532 (3)	total: 491ms	remaining: 20m 27s
4:	test: 0.6963929	test1: 0.6905936	best: 0.6911532 (3)	total: 664ms	remaining: 22m 6s
5:	test: 0.6972963	test1: 0.6910807	best: 0.6911532 (3)	total: 785ms	remaining: 21m 47s
6:	test: 0.6983728	test1: 0.6922826	best: 0.6922826 (6)	total: 894ms	remaining: 21m 15s
7:	test: 0.6993058	test1: 0.6915375	best: 0.6922826 (6)	total: 1s	remaining: 20m 54s
8:	test: 0.7004685	test1: 0.6931749	best: 0.6931749 (8)	total: 1.12s	remaining: 20m 43s
9:	test: 0.7012968	test1: 0.6940679	best: 0.6940679 (9)	total: 1.26s	remaining: 20m 55s
10:	test: 0.7016925	test1: 0.6944245	best: 0.6944245 (10)	total: 1.46s	remaining: 22m 5s
11:	test: 0.7029958	test1: 0.695697

92:	test: 0.7326469	test1: 0.7157031	best: 0.7157031 (92)	total: 13.7s	remaining: 24m 21s
93:	test: 0.7327257	test1: 0.7157942	best: 0.7157942 (93)	total: 14s	remaining: 24m 31s
94:	test: 0.7329567	test1: 0.7159297	best: 0.7159297 (94)	total: 14.2s	remaining: 24m 35s
95:	test: 0.7330442	test1: 0.7160423	best: 0.7160423 (95)	total: 14.3s	remaining: 24m 40s
96:	test: 0.7332392	test1: 0.7161206	best: 0.7161206 (96)	total: 14.5s	remaining: 24m 35s
97:	test: 0.7333567	test1: 0.7161458	best: 0.7161458 (97)	total: 14.6s	remaining: 24m 30s
98:	test: 0.7333935	test1: 0.7162772	best: 0.7162772 (98)	total: 14.7s	remaining: 24m 25s
99:	test: 0.7337405	test1: 0.7166120	best: 0.7166120 (99)	total: 14.8s	remaining: 24m 20s
100:	test: 0.7339511	test1: 0.7166639	best: 0.7166639 (100)	total: 14.9s	remaining: 24m 18s
101:	test: 0.7341124	test1: 0.7167871	best: 0.7167871 (101)	total: 15s	remaining: 24m 16s
102:	test: 0.7344121	test1: 0.7168489	best: 0.7168489 (102)	total: 15.1s	remaining: 24m 13s
103:	tes

183:	test: 0.7488061	test1: 0.7191635	best: 0.7191885 (181)	total: 23.7s	remaining: 21m 3s
184:	test: 0.7488897	test1: 0.7191871	best: 0.7191885 (181)	total: 23.8s	remaining: 21m 1s
185:	test: 0.7490890	test1: 0.7192087	best: 0.7192087 (185)	total: 23.9s	remaining: 20m 59s
186:	test: 0.7493446	test1: 0.7191140	best: 0.7192087 (185)	total: 24s	remaining: 20m 59s
187:	test: 0.7494535	test1: 0.7191493	best: 0.7192087 (185)	total: 24.1s	remaining: 20m 57s
188:	test: 0.7496972	test1: 0.7191999	best: 0.7192087 (185)	total: 24.2s	remaining: 20m 55s
189:	test: 0.7498977	test1: 0.7192663	best: 0.7192663 (189)	total: 24.3s	remaining: 20m 55s
190:	test: 0.7500227	test1: 0.7193423	best: 0.7193423 (190)	total: 24.4s	remaining: 20m 53s
191:	test: 0.7501934	test1: 0.7194226	best: 0.7194226 (191)	total: 24.5s	remaining: 20m 51s
192:	test: 0.7503040	test1: 0.7193902	best: 0.7194226 (191)	total: 24.6s	remaining: 20m 51s
193:	test: 0.7504606	test1: 0.7194366	best: 0.7194366 (193)	total: 24.7s	remaining: 

274:	test: 0.7632276	test1: 0.7200974	best: 0.7201977 (259)	total: 33.1s	remaining: 19m 30s
275:	test: 0.7633081	test1: 0.7201089	best: 0.7201977 (259)	total: 33.2s	remaining: 19m 29s
276:	test: 0.7634110	test1: 0.7201516	best: 0.7201977 (259)	total: 33.3s	remaining: 19m 28s
277:	test: 0.7636057	test1: 0.7202359	best: 0.7202359 (277)	total: 33.4s	remaining: 19m 28s
278:	test: 0.7636969	test1: 0.7202493	best: 0.7202493 (278)	total: 33.5s	remaining: 19m 27s
279:	test: 0.7637799	test1: 0.7202579	best: 0.7202579 (279)	total: 33.6s	remaining: 19m 26s
280:	test: 0.7640236	test1: 0.7203211	best: 0.7203211 (280)	total: 33.7s	remaining: 19m 26s
281:	test: 0.7641024	test1: 0.7203515	best: 0.7203515 (281)	total: 33.8s	remaining: 19m 26s
282:	test: 0.7642053	test1: 0.7204647	best: 0.7204647 (282)	total: 33.9s	remaining: 19m 25s
283:	test: 0.7642728	test1: 0.7204770	best: 0.7204770 (283)	total: 34s	remaining: 19m 24s
284:	test: 0.7644881	test1: 0.7205010	best: 0.7205010 (284)	total: 34.2s	remaining

366:	test: 0.7757048	test1: 0.7214520	best: 0.7215900 (362)	total: 42.6s	remaining: 18m 38s
367:	test: 0.7758818	test1: 0.7213340	best: 0.7215900 (362)	total: 42.7s	remaining: 18m 37s
368:	test: 0.7759622	test1: 0.7213120	best: 0.7215900 (362)	total: 42.8s	remaining: 18m 37s
369:	test: 0.7760511	test1: 0.7212535	best: 0.7215900 (362)	total: 42.9s	remaining: 18m 36s
370:	test: 0.7762944	test1: 0.7213355	best: 0.7215900 (362)	total: 43s	remaining: 18m 36s
371:	test: 0.7764404	test1: 0.7214352	best: 0.7215900 (362)	total: 43.1s	remaining: 18m 36s
372:	test: 0.7766107	test1: 0.7214563	best: 0.7215900 (362)	total: 43.2s	remaining: 18m 36s
373:	test: 0.7767597	test1: 0.7215549	best: 0.7215900 (362)	total: 43.3s	remaining: 18m 35s
374:	test: 0.7769071	test1: 0.7212918	best: 0.7215900 (362)	total: 43.5s	remaining: 18m 35s
375:	test: 0.7769566	test1: 0.7213213	best: 0.7215900 (362)	total: 43.6s	remaining: 18m 35s
376:	test: 0.7770707	test1: 0.7214853	best: 0.7215900 (362)	total: 43.7s	remaining

...

184:	test: 0.7476886	test1: 0.7269125	best: 0.7272237 (167)	total: 19.6s	remaining: 17m 18s
185:	test: 0.7478662	test1: 0.7265867	best: 0.7272237 (167)	total: 19.7s	remaining: 17m 18s
186:	test: 0.7479487	test1: 0.7265819	best: 0.7272237 (167)	total: 19.8s	remaining: 17m 17s
187:	test: 0.7480866	test1: 0.7265079	best: 0.7272237 (167)	total: 19.9s	remaining: 17m 17s
188:	test: 0.7481301	test1: 0.7264441	best: 0.7272237 (167)	total: 20s	remaining: 17m 17s
189:	test: 0.7482435	test1: 0.7264147	best: 0.7272237 (167)	total: 20.1s	remaining: 17m 16s
190:	test: 0.7485359	test1: 0.7263695	best: 0.7272237 (167)	total: 20.2s	remaining: 17m 17s
191:	test: 0.7485483	test1: 0.7263493	best: 0.7272237 (167)	total: 20.3s	remaining: 17m 16s
192:	test: 0.7486596	test1: 0.7263412	best: 0.7272237 (167)	total: 20.4s	remaining: 17m 15s
193:	test: 0.7488225	test1: 0.7263941	best: 0.7272237 (167)	total: 20.5s	remaining: 17m 16s
194:	test: 0.7488999	test1: 0.7263853	best: 0.7272237 (167)	total: 20.6s	remaining

  3%|▎         | 3/99 [00:00<00:04, 22.52it/s]

Fold 4, Valid score = 0.72722


100%|██████████| 99/99 [00:04<00:00, 21.84it/s]

Score by each fold: [0.72159, 0.73148, 0.72993, 0.71731, 0.72722]
Mean scores 0.725506


In [9]:
estimators_arr_m, oof_pred_arr_m, features_m = pf.validation(
    main_x, 
    main_y, 
    cv_strategy, 
    ctb.CatBoostClassifier, 
    PARAM_LGB, 
    roc_auc_score,
    categorial,
    features
)

0:	test: 0.6192837	test1: 0.6248405	best: 0.6248405 (0)	total: 65.9ms	remaining: 10m 59s
1:	test: 0.6842422	test1: 0.6773364	best: 0.6773364 (1)	total: 148ms	remaining: 12m 21s
2:	test: 0.6914637	test1: 0.6841485	best: 0.6841485 (2)	total: 258ms	remaining: 14m 20s
3:	test: 0.6947622	test1: 0.6860857	best: 0.6860857 (3)	total: 377ms	remaining: 15m 42s
4:	test: 0.6984886	test1: 0.6899734	best: 0.6899734 (4)	total: 464ms	remaining: 15m 27s
5:	test: 0.7004906	test1: 0.6910785	best: 0.6910785 (5)	total: 558ms	remaining: 15m 28s
6:	test: 0.7007521	test1: 0.6925212	best: 0.6925212 (6)	total: 665ms	remaining: 15m 49s
7:	test: 0.7011088	test1: 0.6922724	best: 0.6925212 (6)	total: 754ms	remaining: 15m 42s
8:	test: 0.7013225	test1: 0.6925388	best: 0.6925388 (8)	total: 904ms	remaining: 16m 43s
9:	test: 0.7016832	test1: 0.6933620	best: 0.6933620 (9)	total: 980ms	remaining: 16m 18s
10:	test: 0.7018097	test1: 0.6941289	best: 0.6941289 (10)	total: 1.19s	remaining: 17m 58s
11:	test: 0.7030452	test1: 0.

92:	test: 0.7335037	test1: 0.7171740	best: 0.7172622 (89)	total: 9.75s	remaining: 17m 18s
93:	test: 0.7337803	test1: 0.7173024	best: 0.7173024 (93)	total: 9.84s	remaining: 17m 16s
94:	test: 0.7339540	test1: 0.7175076	best: 0.7175076 (94)	total: 9.93s	remaining: 17m 15s
95:	test: 0.7340543	test1: 0.7176293	best: 0.7176293 (95)	total: 10s	remaining: 17m 15s
96:	test: 0.7342461	test1: 0.7176204	best: 0.7176293 (95)	total: 10.1s	remaining: 17m 14s
97:	test: 0.7344322	test1: 0.7177191	best: 0.7177191 (97)	total: 10.2s	remaining: 17m 12s
98:	test: 0.7346566	test1: 0.7177885	best: 0.7177885 (98)	total: 10.3s	remaining: 17m 12s
99:	test: 0.7348363	test1: 0.7178491	best: 0.7178491 (99)	total: 10.4s	remaining: 17m 11s
100:	test: 0.7349607	test1: 0.7177190	best: 0.7178491 (99)	total: 10.5s	remaining: 17m 8s
101:	test: 0.7351068	test1: 0.7174771	best: 0.7178491 (99)	total: 10.6s	remaining: 17m 8s
102:	test: 0.7352479	test1: 0.7174330	best: 0.7178491 (99)	total: 10.7s	remaining: 17m 6s
103:	test: 0

182:	test: 0.7500984	test1: 0.7210108	best: 0.7210409 (181)	total: 18.3s	remaining: 16m 23s
183:	test: 0.7503393	test1: 0.7210367	best: 0.7210409 (181)	total: 18.4s	remaining: 16m 22s
184:	test: 0.7505453	test1: 0.7209836	best: 0.7210409 (181)	total: 18.5s	remaining: 16m 21s
185:	test: 0.7507060	test1: 0.7210270	best: 0.7210409 (181)	total: 18.6s	remaining: 16m 22s
186:	test: 0.7508509	test1: 0.7209058	best: 0.7210409 (181)	total: 18.7s	remaining: 16m 21s
187:	test: 0.7510206	test1: 0.7210163	best: 0.7210409 (181)	total: 18.8s	remaining: 16m 20s
188:	test: 0.7510985	test1: 0.7211499	best: 0.7211499 (188)	total: 18.9s	remaining: 16m 20s
189:	test: 0.7511858	test1: 0.7211686	best: 0.7211686 (189)	total: 19s	remaining: 16m 19s
190:	test: 0.7512789	test1: 0.7211518	best: 0.7211686 (189)	total: 19.1s	remaining: 16m 18s
191:	test: 0.7513760	test1: 0.7212185	best: 0.7212185 (191)	total: 19.2s	remaining: 16m 19s
192:	test: 0.7515441	test1: 0.7211740	best: 0.7212185 (191)	total: 19.3s	remaining

272:	test: 0.7627716	test1: 0.7224928	best: 0.7226080 (263)	total: 26.7s	remaining: 15m 52s
273:	test: 0.7629220	test1: 0.7224433	best: 0.7226080 (263)	total: 26.8s	remaining: 15m 52s
274:	test: 0.7631202	test1: 0.7225702	best: 0.7226080 (263)	total: 26.9s	remaining: 15m 52s
275:	test: 0.7631677	test1: 0.7225525	best: 0.7226080 (263)	total: 27s	remaining: 15m 52s
276:	test: 0.7633551	test1: 0.7225267	best: 0.7226080 (263)	total: 27.1s	remaining: 15m 52s
277:	test: 0.7635141	test1: 0.7226199	best: 0.7226199 (277)	total: 27.2s	remaining: 15m 51s
278:	test: 0.7636097	test1: 0.7226419	best: 0.7226419 (278)	total: 27.3s	remaining: 15m 51s
279:	test: 0.7637442	test1: 0.7227527	best: 0.7227527 (279)	total: 27.4s	remaining: 15m 51s
280:	test: 0.7638053	test1: 0.7227214	best: 0.7227527 (279)	total: 27.5s	remaining: 15m 51s
281:	test: 0.7639997	test1: 0.7226472	best: 0.7227527 (279)	total: 27.6s	remaining: 15m 51s
282:	test: 0.7641000	test1: 0.7226230	best: 0.7227527 (279)	total: 27.7s	remaining

362:	test: 0.7747935	test1: 0.7238477	best: 0.7239480 (355)	total: 35.3s	remaining: 15m 36s
363:	test: 0.7748263	test1: 0.7238252	best: 0.7239480 (355)	total: 35.4s	remaining: 15m 36s
364:	test: 0.7750462	test1: 0.7238755	best: 0.7239480 (355)	total: 35.5s	remaining: 15m 35s
365:	test: 0.7752112	test1: 0.7237751	best: 0.7239480 (355)	total: 35.6s	remaining: 15m 35s
366:	test: 0.7752613	test1: 0.7238964	best: 0.7239480 (355)	total: 35.6s	remaining: 15m 35s
367:	test: 0.7754771	test1: 0.7238807	best: 0.7239480 (355)	total: 35.7s	remaining: 15m 34s
368:	test: 0.7756176	test1: 0.7236879	best: 0.7239480 (355)	total: 35.8s	remaining: 15m 35s
369:	test: 0.7757327	test1: 0.7235497	best: 0.7239480 (355)	total: 35.9s	remaining: 15m 34s
370:	test: 0.7758466	test1: 0.7236578	best: 0.7239480 (355)	total: 36s	remaining: 15m 34s
371:	test: 0.7758930	test1: 0.7237034	best: 0.7239480 (355)	total: 36.1s	remaining: 15m 34s
372:	test: 0.7759440	test1: 0.7238367	best: 0.7239480 (355)	total: 36.2s	remaining

...

117:	test: 0.7400967	test1: 0.7152302	best: 0.7153512 (115)	total: 11.7s	remaining: 16m 20s
118:	test: 0.7402777	test1: 0.7152514	best: 0.7153512 (115)	total: 11.8s	remaining: 16m 19s
119:	test: 0.7405362	test1: 0.7153199	best: 0.7153512 (115)	total: 11.9s	remaining: 16m 18s
120:	test: 0.7406299	test1: 0.7153195	best: 0.7153512 (115)	total: 12s	remaining: 16m 18s
121:	test: 0.7408298	test1: 0.7153475	best: 0.7153512 (115)	total: 12.1s	remaining: 16m 16s
122:	test: 0.7410742	test1: 0.7154402	best: 0.7154402 (122)	total: 12.1s	remaining: 16m 15s
123:	test: 0.7412571	test1: 0.7153799	best: 0.7154402 (122)	total: 12.2s	remaining: 16m 15s
124:	test: 0.7414452	test1: 0.7156764	best: 0.7156764 (124)	total: 12.3s	remaining: 16m 14s
125:	test: 0.7415487	test1: 0.7157903	best: 0.7157903 (125)	total: 12.4s	remaining: 16m 13s
126:	test: 0.7417681	test1: 0.7158914	best: 0.7158914 (126)	total: 12.5s	remaining: 16m 13s
127:	test: 0.7418930	test1: 0.7160646	best: 0.7160646 (127)	total: 12.6s	remaining

207:	test: 0.7530927	test1: 0.7178057	best: 0.7178057 (207)	total: 20.2s	remaining: 15m 48s
208:	test: 0.7532118	test1: 0.7178736	best: 0.7178736 (208)	total: 20.2s	remaining: 15m 48s
209:	test: 0.7535117	test1: 0.7177600	best: 0.7178736 (208)	total: 20.3s	remaining: 15m 48s
210:	test: 0.7536464	test1: 0.7178946	best: 0.7178946 (210)	total: 20.4s	remaining: 15m 48s
211:	test: 0.7537213	test1: 0.7178518	best: 0.7178946 (210)	total: 20.5s	remaining: 15m 48s
212:	test: 0.7538661	test1: 0.7179940	best: 0.7179940 (212)	total: 20.6s	remaining: 15m 47s
213:	test: 0.7539471	test1: 0.7179491	best: 0.7179940 (212)	total: 20.7s	remaining: 15m 47s
214:	test: 0.7541442	test1: 0.7179564	best: 0.7179940 (212)	total: 20.8s	remaining: 15m 46s
215:	test: 0.7542094	test1: 0.7179687	best: 0.7179940 (212)	total: 20.9s	remaining: 15m 46s
216:	test: 0.7543161	test1: 0.7178844	best: 0.7179940 (212)	total: 21s	remaining: 15m 46s
217:	test: 0.7544230	test1: 0.7179740	best: 0.7179940 (212)	total: 21.1s	remaining

18:	test: 0.7094900	test1: 0.7119043	best: 0.7119043 (18)	total: 1.75s	remaining: 15m 20s
19:	test: 0.7099243	test1: 0.7127458	best: 0.7127458 (19)	total: 1.83s	remaining: 15m 14s
20:	test: 0.7102502	test1: 0.7130513	best: 0.7130513 (20)	total: 1.92s	remaining: 15m 10s
21:	test: 0.7104992	test1: 0.7129316	best: 0.7130513 (20)	total: 2.03s	remaining: 15m 20s
22:	test: 0.7113375	test1: 0.7130637	best: 0.7130637 (22)	total: 2.12s	remaining: 15m 19s
23:	test: 0.7110451	test1: 0.7135619	best: 0.7135619 (23)	total: 2.2s	remaining: 15m 16s
24:	test: 0.7115458	test1: 0.7130878	best: 0.7135619 (23)	total: 2.31s	remaining: 15m 22s
25:	test: 0.7117545	test1: 0.7129914	best: 0.7135619 (23)	total: 2.4s	remaining: 15m 19s
26:	test: 0.7120140	test1: 0.7136205	best: 0.7136205 (26)	total: 2.48s	remaining: 15m 17s
27:	test: 0.7126939	test1: 0.7135631	best: 0.7136205 (26)	total: 2.59s	remaining: 15m 23s
28:	test: 0.7130481	test1: 0.7142016	best: 0.7142016 (28)	total: 2.68s	remaining: 15m 21s
29:	test: 0.

110:	test: 0.7363606	test1: 0.7265629	best: 0.7265629 (110)	total: 10.6s	remaining: 15m 45s
111:	test: 0.7366362	test1: 0.7266807	best: 0.7266807 (111)	total: 10.7s	remaining: 15m 44s
112:	test: 0.7367617	test1: 0.7266642	best: 0.7266807 (111)	total: 10.8s	remaining: 15m 43s
113:	test: 0.7369083	test1: 0.7266110	best: 0.7266807 (111)	total: 10.9s	remaining: 15m 45s
114:	test: 0.7371814	test1: 0.7268555	best: 0.7268555 (114)	total: 11s	remaining: 15m 45s
115:	test: 0.7373025	test1: 0.7267460	best: 0.7268555 (114)	total: 11.1s	remaining: 15m 44s
116:	test: 0.7374298	test1: 0.7267122	best: 0.7268555 (114)	total: 11.2s	remaining: 15m 44s
117:	test: 0.7376451	test1: 0.7266920	best: 0.7268555 (114)	total: 11.3s	remaining: 15m 42s
118:	test: 0.7377875	test1: 0.7267246	best: 0.7268555 (114)	total: 11.4s	remaining: 15m 42s
119:	test: 0.7379597	test1: 0.7267238	best: 0.7268555 (114)	total: 11.5s	remaining: 15m 43s
120:	test: 0.7381276	test1: 0.7267603	best: 0.7268555 (114)	total: 11.5s	remaining

200:	test: 0.7506524	test1: 0.7275631	best: 0.7279401 (182)	total: 19s	remaining: 15m 24s
201:	test: 0.7507926	test1: 0.7275047	best: 0.7279401 (182)	total: 19s	remaining: 15m 23s
202:	test: 0.7509913	test1: 0.7275299	best: 0.7279401 (182)	total: 19.1s	remaining: 15m 23s
203:	test: 0.7512054	test1: 0.7275148	best: 0.7279401 (182)	total: 19.2s	remaining: 15m 23s
204:	test: 0.7515168	test1: 0.7275043	best: 0.7279401 (182)	total: 19.3s	remaining: 15m 22s
205:	test: 0.7516261	test1: 0.7275179	best: 0.7279401 (182)	total: 19.4s	remaining: 15m 22s
206:	test: 0.7518416	test1: 0.7276362	best: 0.7279401 (182)	total: 19.5s	remaining: 15m 22s
207:	test: 0.7519158	test1: 0.7275879	best: 0.7279401 (182)	total: 19.6s	remaining: 15m 21s
208:	test: 0.7520408	test1: 0.7276496	best: 0.7279401 (182)	total: 19.7s	remaining: 15m 21s
209:	test: 0.7522985	test1: 0.7277217	best: 0.7279401 (182)	total: 19.8s	remaining: 15m 21s
210:	test: 0.7523909	test1: 0.7276709	best: 0.7279401 (182)	total: 19.9s	remaining: 

In [10]:
model_feat_0 = features[0][features[0] > 0].index.tolist()
model_feat_1 = features[1][features[1] > 0].index.tolist()
model_feat_2 = features[2][features[2] > 0].index.tolist()
model_feat_3 = features[3][features[3] > 0].index.tolist()
model_feat_4 = features[4][features[4] > 0].index.tolist()

model_feat_main = []

for fet in model_feat_0:
    if not fet in model_feat_main:
        model_feat_main.append(fet)
for fet in model_feat_1:
    if not fet in model_feat_main:
        model_feat_main.append(fet)
for fet in model_feat_2:
    if not fet in model_feat_main:
        model_feat_main.append(fet)
for fet in model_feat_3:
    if not fet in model_feat_main:
        model_feat_main.append(fet)
for fet in model_feat_4:
    if not fet in model_feat_main:
        model_feat_main.append(fet)
print(model_feat_main)

['EXTERNAL_SCORING_RATING_MEAN', 'EXTERNAL_SCORING_RATING_mean', 'TOTAL_SALARY', 'EXTERNAL_SCORING_RATING_min', 'ratio_credit_to_annuity', 'ratio_salary_to_age', 'EXTERNAL_SCORING_RATING_2', 'EXTERNAL_SCORING_WEIGHT', 'EXTERNAL_SCORING_RATING_nanmedian', 'EXTERNAL_SCORING_RATING_3', 'EDUCATION_LEVEL', 'EXTERNAL_SCORING_RATING_max', 'NAME_CONTRACT_TYPE', 'ratio_credit_to_age', 'EXTERNAL_SCORING_RATING_var', 'loss_6', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMOUNT_CREDIT', 'EXTERNAL_SCORING_RATING_1', 'GENDER', 'loss_3', 'REGION_POPULATION', 'EXISTS_CAR', 'ratio_age_to_job', 'ratio_cat_to_job', 'AGE', 'AMOUNT_ANNUITY', 'ratio_car_to_age', 'loss_2', 'DAYS_CREDIT_min', 'FAMILY_STATUS', 'DAYS_CREDIT_mean', 'ratio_credit_to_salary', 'FLAG_PHONE', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'loss_4', 'ratio_credit_to_job', 'BKI_KURTOSIS', 'BKI_REQUEST_COUNT', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'loss_1', 'ratio_annuity_to_age', 'COUNT_APPLICATION_ACT_1', 'AMT_REQ_CREDIT_BUREAU_DAY', 'FAMILY_SIZE', 'AMT_CREDIT_MAX_OVERDUE'

In [11]:
estimators_arr_main, oof_pred_arr_main, features_main = pf.validation(
    main_x[model_feat_main], 
    main_y, 
    cv_strategy, 
    ctb.CatBoostClassifier, 
    PARAM_LGB, 
    roc_auc_score,
    categorial,
    []
)

0:	test: 0.6407222	test1: 0.6357143	best: 0.6357143 (0)	total: 88.3ms	remaining: 14m 42s
1:	test: 0.6789784	test1: 0.6729164	best: 0.6729164 (1)	total: 184ms	remaining: 15m 22s
2:	test: 0.6864359	test1: 0.6797597	best: 0.6797597 (2)	total: 280ms	remaining: 15m 32s
3:	test: 0.6912544	test1: 0.6854112	best: 0.6854112 (3)	total: 391ms	remaining: 16m 17s
4:	test: 0.6941495	test1: 0.6882249	best: 0.6882249 (4)	total: 485ms	remaining: 16m 9s
5:	test: 0.6940860	test1: 0.6897303	best: 0.6897303 (5)	total: 574ms	remaining: 15m 56s
6:	test: 0.6952832	test1: 0.6902851	best: 0.6902851 (6)	total: 695ms	remaining: 16m 32s
7:	test: 0.6961311	test1: 0.6934659	best: 0.6934659 (7)	total: 788ms	remaining: 16m 24s
8:	test: 0.6991699	test1: 0.6957581	best: 0.6957581 (8)	total: 887ms	remaining: 16m 24s
9:	test: 0.6992544	test1: 0.6960507	best: 0.6960507 (9)	total: 1s	remaining: 16m 39s
10:	test: 0.7005186	test1: 0.6959080	best: 0.6960507 (9)	total: 1.1s	remaining: 16m 42s
11:	test: 0.7013272	test1: 0.695022

94:	test: 0.7339360	test1: 0.7178023	best: 0.7178023 (94)	total: 10.3s	remaining: 17m 53s
95:	test: 0.7344136	test1: 0.7179381	best: 0.7179381 (95)	total: 10.4s	remaining: 17m 52s
96:	test: 0.7347673	test1: 0.7182197	best: 0.7182197 (96)	total: 10.5s	remaining: 17m 51s
97:	test: 0.7349425	test1: 0.7181982	best: 0.7182197 (96)	total: 10.6s	remaining: 17m 52s
98:	test: 0.7351730	test1: 0.7182310	best: 0.7182310 (98)	total: 10.7s	remaining: 17m 51s
99:	test: 0.7353906	test1: 0.7179514	best: 0.7182310 (98)	total: 10.8s	remaining: 17m 49s
100:	test: 0.7354960	test1: 0.7178123	best: 0.7182310 (98)	total: 10.9s	remaining: 17m 50s
101:	test: 0.7357479	test1: 0.7179384	best: 0.7182310 (98)	total: 11s	remaining: 17m 49s
102:	test: 0.7358474	test1: 0.7179527	best: 0.7182310 (98)	total: 11.1s	remaining: 17m 49s
103:	test: 0.7360972	test1: 0.7179035	best: 0.7182310 (98)	total: 11.2s	remaining: 17m 49s
104:	test: 0.7361533	test1: 0.7179844	best: 0.7182310 (98)	total: 11.4s	remaining: 17m 50s
105:	te

185:	test: 0.7494547	test1: 0.7202541	best: 0.7204007 (173)	total: 19.7s	remaining: 17m 17s
186:	test: 0.7495002	test1: 0.7203254	best: 0.7204007 (173)	total: 19.8s	remaining: 17m 16s
187:	test: 0.7495679	test1: 0.7202941	best: 0.7204007 (173)	total: 19.9s	remaining: 17m 16s
188:	test: 0.7497332	test1: 0.7203508	best: 0.7204007 (173)	total: 20s	remaining: 17m 16s
189:	test: 0.7499054	test1: 0.7204032	best: 0.7204032 (189)	total: 20.1s	remaining: 17m 16s
190:	test: 0.7501602	test1: 0.7203441	best: 0.7204032 (189)	total: 20.2s	remaining: 17m 16s
191:	test: 0.7503005	test1: 0.7204105	best: 0.7204105 (191)	total: 20.3s	remaining: 17m 15s
192:	test: 0.7504388	test1: 0.7204572	best: 0.7204572 (192)	total: 20.4s	remaining: 17m 14s
193:	test: 0.7506136	test1: 0.7204771	best: 0.7204771 (193)	total: 20.5s	remaining: 17m 15s
194:	test: 0.7507611	test1: 0.7204446	best: 0.7204771 (193)	total: 20.6s	remaining: 17m 14s
195:	test: 0.7509474	test1: 0.7203911	best: 0.7204771 (193)	total: 20.7s	remaining

278:	test: 0.7632093	test1: 0.7212476	best: 0.7212584 (240)	total: 29.2s	remaining: 16m 58s
279:	test: 0.7634803	test1: 0.7212055	best: 0.7212584 (240)	total: 29.3s	remaining: 16m 57s
280:	test: 0.7635841	test1: 0.7211373	best: 0.7212584 (240)	total: 29.4s	remaining: 16m 56s
281:	test: 0.7637054	test1: 0.7211752	best: 0.7212584 (240)	total: 29.5s	remaining: 16m 57s
282:	test: 0.7637895	test1: 0.7212911	best: 0.7212911 (282)	total: 29.6s	remaining: 16m 57s
283:	test: 0.7639866	test1: 0.7214028	best: 0.7214028 (283)	total: 29.7s	remaining: 16m 57s
284:	test: 0.7640304	test1: 0.7213461	best: 0.7214028 (283)	total: 29.8s	remaining: 16m 57s
285:	test: 0.7642642	test1: 0.7212166	best: 0.7214028 (283)	total: 30s	remaining: 16m 57s
286:	test: 0.7643699	test1: 0.7212102	best: 0.7214028 (283)	total: 30s	remaining: 16m 56s
287:	test: 0.7644681	test1: 0.7212043	best: 0.7214028 (283)	total: 30.1s	remaining: 16m 56s
288:	test: 0.7647354	test1: 0.7212086	best: 0.7214028 (283)	total: 30.2s	remaining: 

  3%|▎         | 3/98 [00:00<00:04, 21.79it/s]

...

184:	test: 0.7472079	test1: 0.7246190	best: 0.7251097 (147)	total: 20.4s	remaining: 17m 59s
185:	test: 0.7472965	test1: 0.7246650	best: 0.7251097 (147)	total: 20.5s	remaining: 17m 59s
186:	test: 0.7473648	test1: 0.7246369	best: 0.7251097 (147)	total: 20.6s	remaining: 17m 58s
187:	test: 0.7474990	test1: 0.7246321	best: 0.7251097 (147)	total: 20.7s	remaining: 17m 58s
188:	test: 0.7476313	test1: 0.7247035	best: 0.7251097 (147)	total: 20.8s	remaining: 17m 58s
189:	test: 0.7477657	test1: 0.7247365	best: 0.7251097 (147)	total: 20.9s	remaining: 17m 57s
190:	test: 0.7480367	test1: 0.7248902	best: 0.7251097 (147)	total: 21s	remaining: 17m 57s
191:	test: 0.7482593	test1: 0.7247730	best: 0.7251097 (147)	total: 21.1s	remaining: 17m 56s
192:	test: 0.7484086	test1: 0.7247286	best: 0.7251097 (147)	total: 21.2s	remaining: 17m 56s
193:	test: 0.7485446	test1: 0.7248397	best: 0.7251097 (147)	total: 21.3s	remaining: 17m 56s
194:	test: 0.7487702	test1: 0.7249369	best: 0.7251097 (147)	total: 21.4s	remaining

  3%|▎         | 3/98 [00:00<00:04, 23.61it/s]

Fold 4, Valid score = 0.72511


100%|██████████| 98/98 [00:04<00:00, 22.32it/s]

Score by each fold: [0.7214, 0.72873, 0.7278, 0.71748, 0.72511]
Mean scores 0.724104


## LGB

In [7]:
estimators_arr_lgb, oof_pred_arr_lgb, features_lgb = pf.validation(
    main_x, 
    main_y, 
    cv_strategy, 
    lgb.LGBMClassifier, 
    PARAM_LGB, 
    roc_auc_score,
    categorial,
    []
)

/home/aleksandr/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	training's binary_logloss: 0.277155	valid_1's binary_logloss: 0.281273
Training until validation scores don't improve for 50 rounds
[2]	training's binary_logloss: 0.27474	valid_1's binary_logloss: 0.279094
[3]	training's binary_logloss: 0.272681	valid_1's binary_logloss: 0.277273
[4]	training's binary_logloss: 0.270854	valid_1's binary_logloss: 0.275658
[5]	training's binary_logloss: 0.269202	valid_1's binary_logloss: 0.274232
[6]	training's binary_logloss: 0.26775	valid_1's binary_logloss: 0.272989
[7]	training's binary_logloss: 0.266429	valid_1's binary_logloss: 0.271868
[8]	training's binary_logloss: 0.265233	valid_1's binary_logloss: 0.270855
[9]	training's binary_logloss: 0.264172	valid_1's binary_logloss: 0.269992
[10]	training's binary_logloss: 0.263192	valid_1's binary_logloss: 0.269171
[11]	training's binary_logloss: 0.26229	valid_1's binary_logloss: 0.26841
[12]	training's binary_logloss: 0.261438	valid_1's binary_logloss: 0.267751
[13]	training's binary_logloss: 0.260651

[109]	training's binary_logloss: 0.236246	valid_1's binary_logloss: 0.257372
[110]	training's binary_logloss: 0.236153	valid_1's binary_logloss: 0.257402
[111]	training's binary_logloss: 0.236005	valid_1's binary_logloss: 0.257429
[112]	training's binary_logloss: 0.235895	valid_1's binary_logloss: 0.257407
[113]	training's binary_logloss: 0.235775	valid_1's binary_logloss: 0.25739
[114]	training's binary_logloss: 0.235675	valid_1's binary_logloss: 0.257411
[115]	training's binary_logloss: 0.235558	valid_1's binary_logloss: 0.257422
[116]	training's binary_logloss: 0.235393	valid_1's binary_logloss: 0.257424
[117]	training's binary_logloss: 0.235204	valid_1's binary_logloss: 0.257448
[118]	training's binary_logloss: 0.235138	valid_1's binary_logloss: 0.257446
[119]	training's binary_logloss: 0.235009	valid_1's binary_logloss: 0.257453
[120]	training's binary_logloss: 0.234891	valid_1's binary_logloss: 0.25745
[121]	training's binary_logloss: 0.234855	valid_1's binary_logloss: 0.257447
[

100%|██████████| 99/99 [00:49<00:00,  1.99it/s]
/home/aleksandr/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	training's binary_logloss: 0.277004	valid_1's binary_logloss: 0.281751
Training until validation scores don't improve for 50 rounds
[2]	training's binary_logloss: 0.274698	valid_1's binary_logloss: 0.279566
[3]	training's binary_logloss: 0.272631	valid_1's binary_logloss: 0.277648
[4]	training's binary_logloss: 0.270808	valid_1's binary_logloss: 0.275975
[5]	training's binary_logloss: 0.269184	valid_1's binary_logloss: 0.27445
[6]	training's binary_logloss: 0.26775	valid_1's binary_logloss: 0.273128
[7]	training's binary_logloss: 0.266462	valid_1's binary_logloss: 0.271953
[8]	training's binary_logloss: 0.265306	valid_1's binary_logloss: 0.270891
[9]	training's binary_logloss: 0.264204	valid_1's binary_logloss: 0.269878
[10]	training's binary_logloss: 0.263195	valid_1's binary_logloss: 0.268999
[11]	training's binary_logloss: 0.262305	valid_1's binary_logloss: 0.268208
[12]	training's binary_logloss: 0.261457	valid_1's binary_logloss: 0.2675
[13]	training's binary_logloss: 0.260656

...

100%|██████████| 99/99 [00:52<00:00,  1.90it/s]
/home/aleksandr/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	training's binary_logloss: 0.278515	valid_1's binary_logloss: 0.27587
Training until validation scores don't improve for 50 rounds
[2]	training's binary_logloss: 0.276104	valid_1's binary_logloss: 0.273769
[3]	training's binary_logloss: 0.274015	valid_1's binary_logloss: 0.272022
[4]	training's binary_logloss: 0.272184	valid_1's binary_logloss: 0.270462
[5]	training's binary_logloss: 0.270573	valid_1's binary_logloss: 0.269045
[6]	training's binary_logloss: 0.269114	valid_1's binary_logloss: 0.267747
[7]	training's binary_logloss: 0.267824	valid_1's binary_logloss: 0.266601
[8]	training's binary_logloss: 0.266631	valid_1's binary_logloss: 0.265554
[9]	training's binary_logloss: 0.265554	valid_1's binary_logloss: 0.264652
[10]	training's binary_logloss: 0.264563	valid_1's binary_logloss: 0.263768
[11]	training's binary_logloss: 0.263681	valid_1's binary_logloss: 0.263025
[12]	training's binary_logloss: 0.262813	valid_1's binary_logloss: 0.262269
[13]	training's binary_logloss: 0.261

[109]	training's binary_logloss: 0.237137	valid_1's binary_logloss: 0.251326
[110]	training's binary_logloss: 0.236989	valid_1's binary_logloss: 0.251319
[111]	training's binary_logloss: 0.236873	valid_1's binary_logloss: 0.251328
[112]	training's binary_logloss: 0.236739	valid_1's binary_logloss: 0.251353
[113]	training's binary_logloss: 0.236591	valid_1's binary_logloss: 0.251355
[114]	training's binary_logloss: 0.23644	valid_1's binary_logloss: 0.251371
[115]	training's binary_logloss: 0.23628	valid_1's binary_logloss: 0.251411
[116]	training's binary_logloss: 0.236142	valid_1's binary_logloss: 0.251389
[117]	training's binary_logloss: 0.236021	valid_1's binary_logloss: 0.251396
[118]	training's binary_logloss: 0.235908	valid_1's binary_logloss: 0.251401
[119]	training's binary_logloss: 0.235748	valid_1's binary_logloss: 0.251379
[120]	training's binary_logloss: 0.235587	valid_1's binary_logloss: 0.251405
[121]	training's binary_logloss: 0.235434	valid_1's binary_logloss: 0.251409
[

100%|██████████| 99/99 [00:36<00:00,  2.70it/s]

Score by each fold: [0.72409, 0.73104, 0.72947, 0.72008, 0.72804]
Mean scores 0.726544


In [32]:
estimators_arr_lgb_m, oof_pred_arr_lgb_m, features_lgb_m = pf.validation(
    main_x, 
    main_y, 
    cv_strategy, 
    lgb.LGBMClassifier, 
    PARAM_LGB, 
    roc_auc_score,
    categorial,
    features_lgb
)

/home/aleksandr/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	training's binary_logloss: 0.277165	valid_1's binary_logloss: 0.281196
Training until validation scores don't improve for 50 rounds
[2]	training's binary_logloss: 0.274761	valid_1's binary_logloss: 0.27902
[3]	training's binary_logloss: 0.272698	valid_1's binary_logloss: 0.277163
[4]	training's binary_logloss: 0.270862	valid_1's binary_logloss: 0.275554
[5]	training's binary_logloss: 0.269221	valid_1's binary_logloss: 0.274098
[6]	training's binary_logloss: 0.267765	valid_1's binary_logloss: 0.272851
[7]	training's binary_logloss: 0.266441	valid_1's binary_logloss: 0.27169
[8]	training's binary_logloss: 0.265286	valid_1's binary_logloss: 0.27064
[9]	training's binary_logloss: 0.264221	valid_1's binary_logloss: 0.269731
[10]	training's binary_logloss: 0.263217	valid_1's binary_logloss: 0.268893
[11]	training's binary_logloss: 0.262325	valid_1's binary_logloss: 0.268182
[12]	training's binary_logloss: 0.261483	valid_1's binary_logloss: 0.26748
[13]	training's binary_logloss: 0.260693

[111]	training's binary_logloss: 0.237671	valid_1's binary_logloss: 0.257065
[112]	training's binary_logloss: 0.237523	valid_1's binary_logloss: 0.257063
[113]	training's binary_logloss: 0.237355	valid_1's binary_logloss: 0.257084
[114]	training's binary_logloss: 0.237234	valid_1's binary_logloss: 0.257046
[115]	training's binary_logloss: 0.237106	valid_1's binary_logloss: 0.257019
[116]	training's binary_logloss: 0.236991	valid_1's binary_logloss: 0.257
[117]	training's binary_logloss: 0.236886	valid_1's binary_logloss: 0.256958
[118]	training's binary_logloss: 0.236773	valid_1's binary_logloss: 0.256943
[119]	training's binary_logloss: 0.236662	valid_1's binary_logloss: 0.256944
[120]	training's binary_logloss: 0.236555	valid_1's binary_logloss: 0.256959
[121]	training's binary_logloss: 0.236462	valid_1's binary_logloss: 0.256948
[122]	training's binary_logloss: 0.236311	valid_1's binary_logloss: 0.256946
[123]	training's binary_logloss: 0.236245	valid_1's binary_logloss: 0.256963
[1

[219]	training's binary_logloss: 0.226028	valid_1's binary_logloss: 0.256771
[220]	training's binary_logloss: 0.225863	valid_1's binary_logloss: 0.25675
[221]	training's binary_logloss: 0.225744	valid_1's binary_logloss: 0.25677
[222]	training's binary_logloss: 0.22563	valid_1's binary_logloss: 0.256792
[223]	training's binary_logloss: 0.225542	valid_1's binary_logloss: 0.256773
[224]	training's binary_logloss: 0.225466	valid_1's binary_logloss: 0.256763
[225]	training's binary_logloss: 0.225312	valid_1's binary_logloss: 0.256731
[226]	training's binary_logloss: 0.225225	valid_1's binary_logloss: 0.256742
[227]	training's binary_logloss: 0.225072	valid_1's binary_logloss: 0.256747
[228]	training's binary_logloss: 0.224989	valid_1's binary_logloss: 0.256753
[229]	training's binary_logloss: 0.224884	valid_1's binary_logloss: 0.256743
[230]	training's binary_logloss: 0.224827	valid_1's binary_logloss: 0.25677
[231]	training's binary_logloss: 0.224787	valid_1's binary_logloss: 0.256765
[23

/home/aleksandr/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


...

[111]	training's binary_logloss: 0.236762	valid_1's binary_logloss: 0.257239
[112]	training's binary_logloss: 0.236615	valid_1's binary_logloss: 0.257254
[113]	training's binary_logloss: 0.236451	valid_1's binary_logloss: 0.257228
[114]	training's binary_logloss: 0.23637	valid_1's binary_logloss: 0.257251
[115]	training's binary_logloss: 0.23623	valid_1's binary_logloss: 0.257267
[116]	training's binary_logloss: 0.236141	valid_1's binary_logloss: 0.257287
[117]	training's binary_logloss: 0.235936	valid_1's binary_logloss: 0.257209
[118]	training's binary_logloss: 0.235812	valid_1's binary_logloss: 0.257217
[119]	training's binary_logloss: 0.235675	valid_1's binary_logloss: 0.257213
[120]	training's binary_logloss: 0.23555	valid_1's binary_logloss: 0.257219
[121]	training's binary_logloss: 0.235412	valid_1's binary_logloss: 0.257221
[122]	training's binary_logloss: 0.235303	valid_1's binary_logloss: 0.257243
[123]	training's binary_logloss: 0.235181	valid_1's binary_logloss: 0.257246
[1

/home/aleksandr/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	training's binary_logloss: 0.278521	valid_1's binary_logloss: 0.275845
Training until validation scores don't improve for 50 rounds
[2]	training's binary_logloss: 0.276126	valid_1's binary_logloss: 0.273838
[3]	training's binary_logloss: 0.274057	valid_1's binary_logloss: 0.272072
[4]	training's binary_logloss: 0.272256	valid_1's binary_logloss: 0.270503
[5]	training's binary_logloss: 0.270657	valid_1's binary_logloss: 0.269068
[6]	training's binary_logloss: 0.269225	valid_1's binary_logloss: 0.267774
[7]	training's binary_logloss: 0.267922	valid_1's binary_logloss: 0.266717
[8]	training's binary_logloss: 0.266737	valid_1's binary_logloss: 0.265662
[9]	training's binary_logloss: 0.265662	valid_1's binary_logloss: 0.264768
[10]	training's binary_logloss: 0.264683	valid_1's binary_logloss: 0.263932
[11]	training's binary_logloss: 0.263741	valid_1's binary_logloss: 0.263167
[12]	training's binary_logloss: 0.262893	valid_1's binary_logloss: 0.262446
[13]	training's binary_logloss: 0.26

[111]	training's binary_logloss: 0.237964	valid_1's binary_logloss: 0.251202
[112]	training's binary_logloss: 0.237807	valid_1's binary_logloss: 0.251199
[113]	training's binary_logloss: 0.237724	valid_1's binary_logloss: 0.251202
[114]	training's binary_logloss: 0.237608	valid_1's binary_logloss: 0.251199
[115]	training's binary_logloss: 0.237455	valid_1's binary_logloss: 0.251193
[116]	training's binary_logloss: 0.237392	valid_1's binary_logloss: 0.25118
[117]	training's binary_logloss: 0.23727	valid_1's binary_logloss: 0.251175
[118]	training's binary_logloss: 0.237223	valid_1's binary_logloss: 0.251152
[119]	training's binary_logloss: 0.237083	valid_1's binary_logloss: 0.251132
[120]	training's binary_logloss: 0.236973	valid_1's binary_logloss: 0.251133
[121]	training's binary_logloss: 0.236897	valid_1's binary_logloss: 0.251138
[122]	training's binary_logloss: 0.236769	valid_1's binary_logloss: 0.251108
[123]	training's binary_logloss: 0.236636	valid_1's binary_logloss: 0.251076
[

[221]	training's binary_logloss: 0.225799	valid_1's binary_logloss: 0.250951
[222]	training's binary_logloss: 0.225761	valid_1's binary_logloss: 0.250964
[223]	training's binary_logloss: 0.225672	valid_1's binary_logloss: 0.250975
[224]	training's binary_logloss: 0.225563	valid_1's binary_logloss: 0.250971
[225]	training's binary_logloss: 0.225441	valid_1's binary_logloss: 0.250981
[226]	training's binary_logloss: 0.22533	valid_1's binary_logloss: 0.250976
[227]	training's binary_logloss: 0.225193	valid_1's binary_logloss: 0.250996
[228]	training's binary_logloss: 0.225096	valid_1's binary_logloss: 0.25097
[229]	training's binary_logloss: 0.224999	valid_1's binary_logloss: 0.250967
[230]	training's binary_logloss: 0.224873	valid_1's binary_logloss: 0.250957
[231]	training's binary_logloss: 0.224747	valid_1's binary_logloss: 0.250964
[232]	training's binary_logloss: 0.224589	valid_1's binary_logloss: 0.250913
[233]	training's binary_logloss: 0.224507	valid_1's binary_logloss: 0.250922
[

In [17]:
model_feat_0 = features_lgb[0][features_lgb[0] > 0].index.tolist()
model_feat_1 = features_lgb[1][features_lgb[1] > 0].index.tolist()
model_feat_2 = features_lgb[2][features_lgb[2] > 0].index.tolist()
model_feat_3 = features_lgb[3][features_lgb[3] > 0].index.tolist()
model_feat_4 = features_lgb[4][features_lgb[4] > 0].index.tolist()

model_feat_main_lgb = []

for fet in model_feat_0:
    if not fet in model_feat_main_lgb:
        model_feat_main_lgb.append(fet)
for fet in model_feat_1:
    if not fet in model_feat_main_lgb:
        model_feat_main_lgb.append(fet)
for fet in model_feat_2:
    if not fet in model_feat_main_lgb:
        model_feat_main_lgb.append(fet)
for fet in model_feat_3:
    if not fet in model_feat_main_lgb:
        model_feat_main_lgb.append(fet)
for fet in model_feat_4:
    if not fet in model_feat_main_lgb:
        model_feat_main_lgb.append(fet)
print(model_feat_main_lgb)

['EXTERNAL_SCORING_RATING_mean', 'ratio_credit_to_annuity', 'EXTERNAL_SCORING_RATING_MEAN', 'EXTERNAL_SCORING_RATING_3', 'EDUCATION_LEVEL', 'GENDER', 'ratio_car_to_age', 'NAME_CONTRACT_TYPE', 'EXTERNAL_SCORING_RATING_max', 'EXTERNAL_SCORING_RATING_nanmedian', 'TOTAL_SALARY', 'loss_5', 'EXTERNAL_SCORING_RATING_1', 'EXTERNAL_SCORING_WEIGHT', 'AGE', 'AMOUNT_CREDIT', 'ratio_annuity_to_job', 'EXTERNAL_SCORING_PROD', 'loss_4', 'ratio_credit_to_salary', 'ratio_age_to_job', 'loss_6', 'EXTERNAL_SCORING_RATING_min', 'ratio_annuity_to_salary', 'FAMILY_STATUS', 'ratio_salary_to_population', 'AMOUNT_ANNUITY', 'ratio_salary_to_age', 'loss_3', 'EXTERNAL_SCORING_RATING_var', 'RATIO_APPLICATION_ACT_4', 'AMT_CREDIT_MAX_OVERDUE', 'COUNT_APPLICATION_CLS_1', 'REGION_POPULATION', 'ratio_credit_to_age', 'AMT_REQ_CREDIT_BUREAU_QRT', 'loss_1', 'BKI_KURTOSIS', 'BKI_REQUEST_COUNT', 'COUNT_APPLICATION_ACT_1', 'DAYS_CREDIT_max', 'ratio_credit_to_job', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'EXTERNAL_SCORING_RATING_2', 'RAT

In [18]:
estimators_arr_lgb_main, oof_pred_arr_lgb_main, features_lgb_main = pf.validation(
    main_x[model_feat_main_lgb], 
    main_y, 
    cv_strategy, 
    lgb.LGBMClassifier, 
    PARAM_LGB, 
    roc_auc_score,
    categorial,
    []
)

/home/aleksandr/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	training's binary_logloss: 0.277155	valid_1's binary_logloss: 0.281273
Training until validation scores don't improve for 50 rounds
[2]	training's binary_logloss: 0.27474	valid_1's binary_logloss: 0.279094
[3]	training's binary_logloss: 0.272681	valid_1's binary_logloss: 0.277273
[4]	training's binary_logloss: 0.270854	valid_1's binary_logloss: 0.275658
[5]	training's binary_logloss: 0.269202	valid_1's binary_logloss: 0.274232
[6]	training's binary_logloss: 0.26775	valid_1's binary_logloss: 0.272989
[7]	training's binary_logloss: 0.266429	valid_1's binary_logloss: 0.271868
[8]	training's binary_logloss: 0.265233	valid_1's binary_logloss: 0.270855
[9]	training's binary_logloss: 0.264172	valid_1's binary_logloss: 0.269992
[10]	training's binary_logloss: 0.263192	valid_1's binary_logloss: 0.269171
[11]	training's binary_logloss: 0.26229	valid_1's binary_logloss: 0.26841
[12]	training's binary_logloss: 0.261438	valid_1's binary_logloss: 0.267751
[13]	training's binary_logloss: 0.260651

[111]	training's binary_logloss: 0.236043	valid_1's binary_logloss: 0.257206
[112]	training's binary_logloss: 0.235905	valid_1's binary_logloss: 0.257216
[113]	training's binary_logloss: 0.235791	valid_1's binary_logloss: 0.257191
[114]	training's binary_logloss: 0.235632	valid_1's binary_logloss: 0.257185
[115]	training's binary_logloss: 0.235548	valid_1's binary_logloss: 0.257216
[116]	training's binary_logloss: 0.235419	valid_1's binary_logloss: 0.257251
[117]	training's binary_logloss: 0.23527	valid_1's binary_logloss: 0.257254
[118]	training's binary_logloss: 0.23513	valid_1's binary_logloss: 0.257241
[119]	training's binary_logloss: 0.235003	valid_1's binary_logloss: 0.257198
[120]	training's binary_logloss: 0.234916	valid_1's binary_logloss: 0.257211
[121]	training's binary_logloss: 0.2348	valid_1's binary_logloss: 0.257194
[122]	training's binary_logloss: 0.234743	valid_1's binary_logloss: 0.257204
[123]	training's binary_logloss: 0.234642	valid_1's binary_logloss: 0.257215
[12

100%|██████████| 82/82 [00:25<00:00,  3.22it/s]
/home/aleksandr/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	training's binary_logloss: 0.277004	valid_1's binary_logloss: 0.281756
Training until validation scores don't improve for 50 rounds
[2]	training's binary_logloss: 0.274698	valid_1's binary_logloss: 0.279571
[3]	training's binary_logloss: 0.272631	valid_1's binary_logloss: 0.277653
[4]	training's binary_logloss: 0.270808	valid_1's binary_logloss: 0.27598
[5]	training's binary_logloss: 0.269185	valid_1's binary_logloss: 0.274435
[6]	training's binary_logloss: 0.267752	valid_1's binary_logloss: 0.273113
[7]	training's binary_logloss: 0.266464	valid_1's binary_logloss: 0.271938
[8]	training's binary_logloss: 0.265308	valid_1's binary_logloss: 0.270876
[9]	training's binary_logloss: 0.264206	valid_1's binary_logloss: 0.269864
[10]	training's binary_logloss: 0.263195	valid_1's binary_logloss: 0.268986
[11]	training's binary_logloss: 0.262306	valid_1's binary_logloss: 0.268196
[12]	training's binary_logloss: 0.261459	valid_1's binary_logloss: 0.267482
[13]	training's binary_logloss: 0.260

...

[1]	training's binary_logloss: 0.278515	valid_1's binary_logloss: 0.27587
Training until validation scores don't improve for 50 rounds
[2]	training's binary_logloss: 0.276104	valid_1's binary_logloss: 0.273769
[3]	training's binary_logloss: 0.274015	valid_1's binary_logloss: 0.272022
[4]	training's binary_logloss: 0.272184	valid_1's binary_logloss: 0.270462
[5]	training's binary_logloss: 0.270573	valid_1's binary_logloss: 0.269045
[6]	training's binary_logloss: 0.269114	valid_1's binary_logloss: 0.267747
[7]	training's binary_logloss: 0.267824	valid_1's binary_logloss: 0.266601
[8]	training's binary_logloss: 0.266631	valid_1's binary_logloss: 0.265554
[9]	training's binary_logloss: 0.265554	valid_1's binary_logloss: 0.264652
[10]	training's binary_logloss: 0.264563	valid_1's binary_logloss: 0.263768
[11]	training's binary_logloss: 0.263681	valid_1's binary_logloss: 0.263025
[12]	training's binary_logloss: 0.262813	valid_1's binary_logloss: 0.262269
[13]	training's binary_logloss: 0.261

[112]	training's binary_logloss: 0.236673	valid_1's binary_logloss: 0.25164
[113]	training's binary_logloss: 0.23651	valid_1's binary_logloss: 0.251632
[114]	training's binary_logloss: 0.236365	valid_1's binary_logloss: 0.2516
[115]	training's binary_logloss: 0.236226	valid_1's binary_logloss: 0.251579
[116]	training's binary_logloss: 0.236072	valid_1's binary_logloss: 0.25157
[117]	training's binary_logloss: 0.235962	valid_1's binary_logloss: 0.25156
[118]	training's binary_logloss: 0.235814	valid_1's binary_logloss: 0.251569
[119]	training's binary_logloss: 0.235691	valid_1's binary_logloss: 0.251574
[120]	training's binary_logloss: 0.235559	valid_1's binary_logloss: 0.251571
[121]	training's binary_logloss: 0.235438	valid_1's binary_logloss: 0.251573
[122]	training's binary_logloss: 0.235282	valid_1's binary_logloss: 0.251575
[123]	training's binary_logloss: 0.23511	valid_1's binary_logloss: 0.251581
[124]	training's binary_logloss: 0.23502	valid_1's binary_logloss: 0.251566
[125]	t

[222]	training's binary_logloss: 0.223611	valid_1's binary_logloss: 0.251667
[223]	training's binary_logloss: 0.223485	valid_1's binary_logloss: 0.251632
[224]	training's binary_logloss: 0.223396	valid_1's binary_logloss: 0.251651
[225]	training's binary_logloss: 0.223292	valid_1's binary_logloss: 0.251626
[226]	training's binary_logloss: 0.223188	valid_1's binary_logloss: 0.251631
[227]	training's binary_logloss: 0.223051	valid_1's binary_logloss: 0.251644
[228]	training's binary_logloss: 0.223001	valid_1's binary_logloss: 0.251643
[229]	training's binary_logloss: 0.222873	valid_1's binary_logloss: 0.251643
[230]	training's binary_logloss: 0.22275	valid_1's binary_logloss: 0.251645
[231]	training's binary_logloss: 0.222667	valid_1's binary_logloss: 0.251682
[232]	training's binary_logloss: 0.222568	valid_1's binary_logloss: 0.251678
[233]	training's binary_logloss: 0.222519	valid_1's binary_logloss: 0.251657
[234]	training's binary_logloss: 0.222406	valid_1's binary_logloss: 0.251635


100%|██████████| 82/82 [00:29<00:00,  2.77it/s]

Score by each fold: [0.72484, 0.73075, 0.72974, 0.72016, 0.72678]
Mean scores 0.726454


## Предсказания на тестовой выборке

In [24]:
df_test = create_data_set(pd.read_csv(PATH + 'test.csv'), PATH)

test_id = df_test[ID]
test_x = df_test.drop([ID], axis=1)

/home/aleksandr/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)


In [25]:
def predict_test(i_test_df: pd.DataFrame, 
                 i_models, 
                 i_features,
                 count_models: int) -> list:
    y_pred = np.zeros(i_test_df.shape[0])
    i = 0
    for est in i_models:
        model_feat = i_features[i][i_features[i] > 0].index.tolist()
        y_pred += est.predict_proba(i_test_df[model_feat])[:, 1]
        i += 1
    y_pred = y_pred/count_models
    return y_pred

In [26]:
def save_predict(predict_values: list, record_id: list, name_columns: list, save_path: str):
    test_pred_df = pd.DataFrame(predict_values, index=record_id, columns=name_columns)
    test_pred_df.to_csv(save_path)

In [54]:
test_predict_values = predict_test(test_x, 
                                   estimators_arr_lgb_m,
                                   features_lgb,
                                   cv_strategy.n_splits)

In [55]:
save_predict(test_predict_values,
             test_id,
             [TARGET],
             PATH + 'test_predict_lgb_model_1.csv'
            )

In [10]:
from sklearn.model_selection import GridSearchCV

In [ ]:
PARAM_LGB = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'n_estimators': 10000,
    'learning_rate': 0.05134,
    'early_stopping_rounds': 50,
    'reg_lambda': 0.05,
    'max_depth': 6,
    'random_state': 24
}

In [8]:
parametrs = {'loss_function': ['Logloss'],
             'eval_metric': ['AUC'],
             'n_estimators': range(100, 500, 100),
             'max_depth': range (1,13, 2),
             'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.09],
#              'early_stopping_rounds': range (10,90, 20),
             'reg_lambda': [0.01, 0.03, 0.05, 0.07, 0.09],
             'random_state': [24]
            }

In [11]:
grid_lgb = lgb.LGBMClassifier()
grid = GridSearchCV(grid_lgb, parametrs, cv=5)
grid.fit(main_x[features_lgb[1].index], main_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_depth=-1,
                                      min_child_samples=20,
                                      min_child_weight=0.001,
                                      min_split_gain=0.0, n_estimators=100,
                                      n_jobs=-1, num_leaves=31, objective=None,
                                      random_state=None, reg_alpha=0.0,
                                      reg_lambda=0.0, silent=Tru...
                                      subsample_freq=0),
             iid='deprecated', n_jobs=None,
             param_grid={'eval_metric': ['AUC'],
                         'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.09],
                         'loss_function': ['Loglo

In [12]:
grid.best_params_

{'eval_metric': 'AUC',
 'learning_rate': 0.07,
 'loss_function': 'Logloss',
 'max_depth': 7,
 'n_estimators': 100,
 'random_state': 24,
 'reg_lambda': 0.03}

In [29]:
param_good = {'eval_metric': 'AUC',
 'learning_rate': 0.03,
 'loss_function': 'Logloss',
 'max_depth': 7,
 'early_stopping_rounds': 50,
 'n_estimators': 1000,
 'random_state': 24,
 'reg_lambda': 0.03}

estimators_arr_lgb_m, oof_pred_arr_lgb_m, features_lgb_m = pf.validation(
    main_x, 
    main_y, 
    cv_strategy, 
    lgb.LGBMClassifier, 
    param_good, 
    roc_auc_score,
    categorial,
    features_lgb
)
# Score Liderboard - 0.73207

/home/aleksandr/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	training's binary_logloss: 0.278305	valid_1's binary_logloss: 0.28231
Training until validation scores don't improve for 50 rounds
[2]	training's binary_logloss: 0.276796	valid_1's binary_logloss: 0.28095
[3]	training's binary_logloss: 0.275386	valid_1's binary_logloss: 0.279721
[4]	training's binary_logloss: 0.274094	valid_1's binary_logloss: 0.278568
[5]	training's binary_logloss: 0.272923	valid_1's binary_logloss: 0.27754
[6]	training's binary_logloss: 0.271814	valid_1's binary_logloss: 0.276579
[7]	training's binary_logloss: 0.270787	valid_1's binary_logloss: 0.275694
[8]	training's binary_logloss: 0.269812	valid_1's binary_logloss: 0.274843
[9]	training's binary_logloss: 0.268899	valid_1's binary_logloss: 0.274045
[10]	training's binary_logloss: 0.268051	valid_1's binary_logloss: 0.273325
[11]	training's binary_logloss: 0.267257	valid_1's binary_logloss: 0.272661
[12]	training's binary_logloss: 0.266513	valid_1's binary_logloss: 0.272024
[13]	training's binary_logloss: 0.26579

[111]	training's binary_logloss: 0.24321	valid_1's binary_logloss: 0.258006
[112]	training's binary_logloss: 0.243092	valid_1's binary_logloss: 0.257971
[113]	training's binary_logloss: 0.242949	valid_1's binary_logloss: 0.25792
[114]	training's binary_logloss: 0.242828	valid_1's binary_logloss: 0.257905
[115]	training's binary_logloss: 0.242729	valid_1's binary_logloss: 0.257888
[116]	training's binary_logloss: 0.242631	valid_1's binary_logloss: 0.25787
[117]	training's binary_logloss: 0.242498	valid_1's binary_logloss: 0.257822
[118]	training's binary_logloss: 0.242385	valid_1's binary_logloss: 0.257811
[119]	training's binary_logloss: 0.242267	valid_1's binary_logloss: 0.257784
[120]	training's binary_logloss: 0.242147	valid_1's binary_logloss: 0.257778
[121]	training's binary_logloss: 0.242041	valid_1's binary_logloss: 0.257761
[122]	training's binary_logloss: 0.241916	valid_1's binary_logloss: 0.257729
[123]	training's binary_logloss: 0.241823	valid_1's binary_logloss: 0.25771
[12

[217]	training's binary_logloss: 0.233344	valid_1's binary_logloss: 0.257031
[218]	training's binary_logloss: 0.233278	valid_1's binary_logloss: 0.257032
[219]	training's binary_logloss: 0.2332	valid_1's binary_logloss: 0.257021
[220]	training's binary_logloss: 0.233115	valid_1's binary_logloss: 0.257007
[221]	training's binary_logloss: 0.233026	valid_1's binary_logloss: 0.257009
[222]	training's binary_logloss: 0.232943	valid_1's binary_logloss: 0.256976
[223]	training's binary_logloss: 0.232877	valid_1's binary_logloss: 0.256987
[224]	training's binary_logloss: 0.232793	valid_1's binary_logloss: 0.256986
[225]	training's binary_logloss: 0.232711	valid_1's binary_logloss: 0.256986
[226]	training's binary_logloss: 0.232622	valid_1's binary_logloss: 0.256999
[227]	training's binary_logloss: 0.232537	valid_1's binary_logloss: 0.257008
[228]	training's binary_logloss: 0.232469	valid_1's binary_logloss: 0.257013
[229]	training's binary_logloss: 0.23239	valid_1's binary_logloss: 0.257008
[2

/home/aleksandr/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


...

/home/aleksandr/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	training's binary_logloss: 0.279654	valid_1's binary_logloss: 0.276874
Training until validation scores don't improve for 50 rounds
[2]	training's binary_logloss: 0.278123	valid_1's binary_logloss: 0.2756
[3]	training's binary_logloss: 0.276717	valid_1's binary_logloss: 0.274424
[4]	training's binary_logloss: 0.275452	valid_1's binary_logloss: 0.273352
[5]	training's binary_logloss: 0.274261	valid_1's binary_logloss: 0.272324
[6]	training's binary_logloss: 0.273185	valid_1's binary_logloss: 0.271388
[7]	training's binary_logloss: 0.272164	valid_1's binary_logloss: 0.270507
[8]	training's binary_logloss: 0.271209	valid_1's binary_logloss: 0.26965
[9]	training's binary_logloss: 0.27032	valid_1's binary_logloss: 0.268863
[10]	training's binary_logloss: 0.269459	valid_1's binary_logloss: 0.268083
[11]	training's binary_logloss: 0.268651	valid_1's binary_logloss: 0.26737
[12]	training's binary_logloss: 0.26791	valid_1's binary_logloss: 0.266736
[13]	training's binary_logloss: 0.267199	v

[109]	training's binary_logloss: 0.244865	valid_1's binary_logloss: 0.25237
[110]	training's binary_logloss: 0.244748	valid_1's binary_logloss: 0.252337
[111]	training's binary_logloss: 0.24464	valid_1's binary_logloss: 0.252311
[112]	training's binary_logloss: 0.244522	valid_1's binary_logloss: 0.252247
[113]	training's binary_logloss: 0.244404	valid_1's binary_logloss: 0.252218
[114]	training's binary_logloss: 0.244293	valid_1's binary_logloss: 0.252194
[115]	training's binary_logloss: 0.244183	valid_1's binary_logloss: 0.252167
[116]	training's binary_logloss: 0.244073	valid_1's binary_logloss: 0.252151
[117]	training's binary_logloss: 0.243953	valid_1's binary_logloss: 0.252085
[118]	training's binary_logloss: 0.243841	valid_1's binary_logloss: 0.25205
[119]	training's binary_logloss: 0.243743	valid_1's binary_logloss: 0.252034
[120]	training's binary_logloss: 0.243619	valid_1's binary_logloss: 0.252015
[121]	training's binary_logloss: 0.243521	valid_1's binary_logloss: 0.252005
[1

[216]	training's binary_logloss: 0.23458	valid_1's binary_logloss: 0.251149
[217]	training's binary_logloss: 0.234509	valid_1's binary_logloss: 0.251138
[218]	training's binary_logloss: 0.234428	valid_1's binary_logloss: 0.251134
[219]	training's binary_logloss: 0.234344	valid_1's binary_logloss: 0.251139
[220]	training's binary_logloss: 0.234256	valid_1's binary_logloss: 0.251126
[221]	training's binary_logloss: 0.234216	valid_1's binary_logloss: 0.251136
[222]	training's binary_logloss: 0.234122	valid_1's binary_logloss: 0.251125
[223]	training's binary_logloss: 0.234043	valid_1's binary_logloss: 0.251144
[224]	training's binary_logloss: 0.233978	valid_1's binary_logloss: 0.251149
[225]	training's binary_logloss: 0.233907	valid_1's binary_logloss: 0.251151
[226]	training's binary_logloss: 0.233812	valid_1's binary_logloss: 0.251163
[227]	training's binary_logloss: 0.233707	valid_1's binary_logloss: 0.251141
[228]	training's binary_logloss: 0.233649	valid_1's binary_logloss: 0.251148


[326]	training's binary_logloss: 0.226558	valid_1's binary_logloss: 0.251116
[327]	training's binary_logloss: 0.226467	valid_1's binary_logloss: 0.251128
[328]	training's binary_logloss: 0.226396	valid_1's binary_logloss: 0.25112
Early stopping, best iteration is:
[278]	training's binary_logloss: 0.229862	valid_1's binary_logloss: 0.251075
Fold 4, Valid score = 0.72841
Score by each fold: [0.72677, 0.73231, 0.72894, 0.72267, 0.72841]
Mean scores 0.72782


In [30]:
test_predict_values = predict_test(test_x, 
                                   estimators_arr_lgb_m,
                                   features_lgb,
                                   cv_strategy.n_splits)
save_predict(test_predict_values,
             test_id,
             [TARGET],
             PATH + 'test_predict_lgb_model_grid_searc1.csv'
            )

In [31]:
param_good = {'eval_metric': 'AUC',
 'learning_rate': 0.01,
 'loss_function': 'Logloss',
 'max_depth': 7,
 'early_stopping_rounds': 100,
 'n_estimators': 1000,
 'random_state': 24,
 'reg_lambda': 0.03}

estimators_arr_lgb_m, oof_pred_arr_lgb_m, features_lgb_m = pf.validation(
    main_x, 
    main_y, 
    cv_strategy, 
    lgb.LGBMClassifier, 
    param_good, 
    roc_auc_score,
    categorial,
    features_lgb
)
# Score Liderboard - 0.73213

/home/aleksandr/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	training's binary_logloss: 0.279423	valid_1's binary_logloss: 0.283299
Training until validation scores don't improve for 100 rounds
[2]	training's binary_logloss: 0.278871	valid_1's binary_logloss: 0.282799
[3]	training's binary_logloss: 0.278348	valid_1's binary_logloss: 0.282329
[4]	training's binary_logloss: 0.277828	valid_1's binary_logloss: 0.281869
[5]	training's binary_logloss: 0.277336	valid_1's binary_logloss: 0.281424
[6]	training's binary_logloss: 0.276856	valid_1's binary_logloss: 0.280988
[7]	training's binary_logloss: 0.276387	valid_1's binary_logloss: 0.280575
[8]	training's binary_logloss: 0.275928	valid_1's binary_logloss: 0.280172
[9]	training's binary_logloss: 0.275486	valid_1's binary_logloss: 0.279791
[10]	training's binary_logloss: 0.275043	valid_1's binary_logloss: 0.279398
[11]	training's binary_logloss: 0.274616	valid_1's binary_logloss: 0.279009
[12]	training's binary_logloss: 0.274207	valid_1's binary_logloss: 0.278649
[13]	training's binary_logloss: 0.2

[112]	training's binary_logloss: 0.255375	valid_1's binary_logloss: 0.263161
[113]	training's binary_logloss: 0.255286	valid_1's binary_logloss: 0.263103
[114]	training's binary_logloss: 0.255191	valid_1's binary_logloss: 0.263038
[115]	training's binary_logloss: 0.2551	valid_1's binary_logloss: 0.262973
[116]	training's binary_logloss: 0.255017	valid_1's binary_logloss: 0.26292
[117]	training's binary_logloss: 0.254929	valid_1's binary_logloss: 0.262865
[118]	training's binary_logloss: 0.254843	valid_1's binary_logloss: 0.26282
[119]	training's binary_logloss: 0.254759	valid_1's binary_logloss: 0.262773
[120]	training's binary_logloss: 0.254672	valid_1's binary_logloss: 0.262716
[121]	training's binary_logloss: 0.254587	valid_1's binary_logloss: 0.262665
[122]	training's binary_logloss: 0.254496	valid_1's binary_logloss: 0.262599
[123]	training's binary_logloss: 0.254409	valid_1's binary_logloss: 0.26254
[124]	training's binary_logloss: 0.254321	valid_1's binary_logloss: 0.262477
[125

[222]	training's binary_logloss: 0.248201	valid_1's binary_logloss: 0.259481
[223]	training's binary_logloss: 0.248151	valid_1's binary_logloss: 0.259464
[224]	training's binary_logloss: 0.248095	valid_1's binary_logloss: 0.259436
[225]	training's binary_logloss: 0.248048	valid_1's binary_logloss: 0.259417
[226]	training's binary_logloss: 0.247993	valid_1's binary_logloss: 0.25939
[227]	training's binary_logloss: 0.247945	valid_1's binary_logloss: 0.259373
[228]	training's binary_logloss: 0.247891	valid_1's binary_logloss: 0.259349
[229]	training's binary_logloss: 0.247845	valid_1's binary_logloss: 0.259332
[230]	training's binary_logloss: 0.247791	valid_1's binary_logloss: 0.25931
[231]	training's binary_logloss: 0.247744	valid_1's binary_logloss: 0.259292
[232]	training's binary_logloss: 0.247698	valid_1's binary_logloss: 0.259277
[233]	training's binary_logloss: 0.247645	valid_1's binary_logloss: 0.259256
[234]	training's binary_logloss: 0.247601	valid_1's binary_logloss: 0.259238
[

[329]	training's binary_logloss: 0.243402	valid_1's binary_logloss: 0.257934
[330]	training's binary_logloss: 0.243367	valid_1's binary_logloss: 0.257931
[331]	training's binary_logloss: 0.243319	valid_1's binary_logloss: 0.257912
[332]	training's binary_logloss: 0.243283	valid_1's binary_logloss: 0.257898
[333]	training's binary_logloss: 0.243235	valid_1's binary_logloss: 0.257899
[334]	training's binary_logloss: 0.243192	valid_1's binary_logloss: 0.257888
[335]	training's binary_logloss: 0.243153	valid_1's binary_logloss: 0.25788
[336]	training's binary_logloss: 0.243116	valid_1's binary_logloss: 0.257871
[337]	training's binary_logloss: 0.243067	valid_1's binary_logloss: 0.257851
[338]	training's binary_logloss: 0.243029	valid_1's binary_logloss: 0.257838
[339]	training's binary_logloss: 0.242989	valid_1's binary_logloss: 0.257826
[340]	training's binary_logloss: 0.242952	valid_1's binary_logloss: 0.257823
[341]	training's binary_logloss: 0.242918	valid_1's binary_logloss: 0.257818


...

[546]	training's binary_logloss: 0.237729	valid_1's binary_logloss: 0.251251
[547]	training's binary_logloss: 0.237701	valid_1's binary_logloss: 0.251255
[548]	training's binary_logloss: 0.237668	valid_1's binary_logloss: 0.251252
[549]	training's binary_logloss: 0.237639	valid_1's binary_logloss: 0.251248
[550]	training's binary_logloss: 0.237612	valid_1's binary_logloss: 0.251244
[551]	training's binary_logloss: 0.237587	valid_1's binary_logloss: 0.251249
[552]	training's binary_logloss: 0.237557	valid_1's binary_logloss: 0.251243
[553]	training's binary_logloss: 0.237519	valid_1's binary_logloss: 0.251236
[554]	training's binary_logloss: 0.237494	valid_1's binary_logloss: 0.251236
[555]	training's binary_logloss: 0.237465	valid_1's binary_logloss: 0.251241
[556]	training's binary_logloss: 0.237439	valid_1's binary_logloss: 0.251239
[557]	training's binary_logloss: 0.237407	valid_1's binary_logloss: 0.251235
[558]	training's binary_logloss: 0.237377	valid_1's binary_logloss: 0.251236

[656]	training's binary_logloss: 0.234759	valid_1's binary_logloss: 0.251071
[657]	training's binary_logloss: 0.234732	valid_1's binary_logloss: 0.251071
[658]	training's binary_logloss: 0.234706	valid_1's binary_logloss: 0.251067
[659]	training's binary_logloss: 0.234679	valid_1's binary_logloss: 0.251067
[660]	training's binary_logloss: 0.23465	valid_1's binary_logloss: 0.251066
[661]	training's binary_logloss: 0.234626	valid_1's binary_logloss: 0.251064
[662]	training's binary_logloss: 0.234601	valid_1's binary_logloss: 0.251066
[663]	training's binary_logloss: 0.234571	valid_1's binary_logloss: 0.251066
[664]	training's binary_logloss: 0.234551	valid_1's binary_logloss: 0.251065
[665]	training's binary_logloss: 0.234529	valid_1's binary_logloss: 0.251066
[666]	training's binary_logloss: 0.234502	valid_1's binary_logloss: 0.251066
[667]	training's binary_logloss: 0.234476	valid_1's binary_logloss: 0.251067
[668]	training's binary_logloss: 0.234455	valid_1's binary_logloss: 0.251064


[767]	training's binary_logloss: 0.231966	valid_1's binary_logloss: 0.250955
[768]	training's binary_logloss: 0.231951	valid_1's binary_logloss: 0.25095
[769]	training's binary_logloss: 0.231923	valid_1's binary_logloss: 0.250955
[770]	training's binary_logloss: 0.231896	valid_1's binary_logloss: 0.250953
[771]	training's binary_logloss: 0.231875	valid_1's binary_logloss: 0.250953
[772]	training's binary_logloss: 0.231846	valid_1's binary_logloss: 0.250949
[773]	training's binary_logloss: 0.231816	valid_1's binary_logloss: 0.250945
[774]	training's binary_logloss: 0.231797	valid_1's binary_logloss: 0.250939
[775]	training's binary_logloss: 0.231766	valid_1's binary_logloss: 0.250938
[776]	training's binary_logloss: 0.231735	valid_1's binary_logloss: 0.250933
[777]	training's binary_logloss: 0.231713	valid_1's binary_logloss: 0.250937
[778]	training's binary_logloss: 0.231686	valid_1's binary_logloss: 0.250942
[779]	training's binary_logloss: 0.231665	valid_1's binary_logloss: 0.250943


[877]	training's binary_logloss: 0.229352	valid_1's binary_logloss: 0.250922
[878]	training's binary_logloss: 0.229327	valid_1's binary_logloss: 0.250922
[879]	training's binary_logloss: 0.229301	valid_1's binary_logloss: 0.250915
[880]	training's binary_logloss: 0.229276	valid_1's binary_logloss: 0.250911
[881]	training's binary_logloss: 0.229248	valid_1's binary_logloss: 0.250918
[882]	training's binary_logloss: 0.229226	valid_1's binary_logloss: 0.25092
[883]	training's binary_logloss: 0.229212	valid_1's binary_logloss: 0.25092
[884]	training's binary_logloss: 0.229184	valid_1's binary_logloss: 0.250921
[885]	training's binary_logloss: 0.229159	valid_1's binary_logloss: 0.250915
[886]	training's binary_logloss: 0.229129	valid_1's binary_logloss: 0.250916
[887]	training's binary_logloss: 0.229104	valid_1's binary_logloss: 0.250914
[888]	training's binary_logloss: 0.229076	valid_1's binary_logloss: 0.250906
[889]	training's binary_logloss: 0.229057	valid_1's binary_logloss: 0.250903
[

[984]	training's binary_logloss: 0.226971	valid_1's binary_logloss: 0.250918
[985]	training's binary_logloss: 0.226947	valid_1's binary_logloss: 0.250922
[986]	training's binary_logloss: 0.226929	valid_1's binary_logloss: 0.250919
[987]	training's binary_logloss: 0.226903	valid_1's binary_logloss: 0.250928
[988]	training's binary_logloss: 0.226881	valid_1's binary_logloss: 0.250923
[989]	training's binary_logloss: 0.226867	valid_1's binary_logloss: 0.250924
[990]	training's binary_logloss: 0.226846	valid_1's binary_logloss: 0.250927
[991]	training's binary_logloss: 0.226818	valid_1's binary_logloss: 0.250925
[992]	training's binary_logloss: 0.226791	valid_1's binary_logloss: 0.250914
[993]	training's binary_logloss: 0.226774	valid_1's binary_logloss: 0.250911
[994]	training's binary_logloss: 0.226744	valid_1's binary_logloss: 0.250908
[995]	training's binary_logloss: 0.226726	valid_1's binary_logloss: 0.250913
[996]	training's binary_logloss: 0.226701	valid_1's binary_logloss: 0.250918

In [32]:
test_predict_values = predict_test(test_x, 
                                   estimators_arr_lgb_m,
                                   features_lgb,
                                   cv_strategy.n_splits)
save_predict(test_predict_values,
             test_id,
             [TARGET],
             PATH + 'test_predict_lgb_model_grid_searc2.csv'
            )